
# Diseño de software para cómputo científico

----

## Unidad 5: Integración con lenguajes de alto nivel con bajo nivel.


## Agenda de la Unidad 5

- JIT (Numba)
- Cython.
- **Integración de Python con FORTRAN.**
- Integración de Python con C. 



## Recapitulando

- Escribimos el código Python.
- Pasamos todo a numpy.
- Hicimos profile.
- Paralelisamos (joblib/dask).
- Hicimos profile.
- Usamos Numba.
- Hicimos profile.
- Si podemos elegir el lenguaje: Cython
    - Si no podemos elegir el lenguaje y vamos a hacer cómputo numérico FORTRAN.


## Fortran

- Previamente FORTRAN, contracción del inglés The IBM Mathematical Formula Translating System.
- Es un lenguaje de programación de alto nivel de propósito general, procedimental e imperativo, que está especialmente adaptado al cálculo numérico y a la computación científica.
- Desarrollado originalmente por IBM en 1957 por John Backus.
- Es una familia de lenguajes los cuales incluyen entre otros: FORTRAN - FORTRAN II - FORTRAN 77 - Fortran 95 - Fortran 2018.

![imgs/apuphd.img](imgs/apuphd.gif)

La tesis de Apu esta hecha probablemente en FORTRAN o algún tipo de Assembler.

### F2PY

- F2PY es una herramienta que permite la comunicación entre Python y Fortran. F2PY genera módulos de extensión (extension modules) para Python a partir de código Fortran, lo que permite:
- Utilizar subrutinas, datos en bloques COMMON y variables en módulos de FORTRAN 77 o Fortran 90/95 desde    Python.
- Llamar funciones de Python desde Fortran (callbacks).
- Manejar automáticamente la diferencia entre arrays NumPy-contiguos (esto es, C-contiguos) y Fortran-contiguos.
- Fue creado en 1999 por Pearu Peterson mientras era estudiante de doctorado en la Universidad Técnica de Tallin, y en 2005 después de varias versiones estables quedó incluido dentro de NumPy.

## Problemas de F2PY

- F2PY funciona con FORTRAN 77 o Fortran 90/95 que no incluya características «modernas» como punteros, tipos derivados o arrays en forma asumida; prueba de ello es  SciPy. 
- El soporte de Fortran 90/95 es incompleto, a veces hay que editar las cabeceras manualmente (ya hablaremos de ellas)
- Para código antiguo, especialmente para el muy antiguo, no hay problemas; para código nuevo y moderno hará falta un poco más de maña, pero el resultado merece la pena.

## Instalación

- Si tienen numpy tienen F2PY.
- Necesitamos tener instalado `gfortran`.
- Ademas vale la pena instalar para usar el *cell-magic* 

```bash
$ pip install fortran-magic
```

In [1]:
!pip install fortran-magic

In [2]:
%load_ext fortranmagic

## Hola mundo en Fortran 95

In [3]:
%%fortran

subroutine hola_mundo(msg)
    character(len=12), intent(out) :: msg
    msg = 'Hola, mundo!'
end subroutine

In [4]:
hola_mundo()

b'Hola, mundo!'

In [5]:
hola_mundo?

## Vectores

```fortran
module vectores
    implicit none
    contains

    ! Producto escalar entre dos vectores u, v de longitud n
    function producto_escalar(n, u, v) result(p)
        integer, intent(in) :: n
        double precision, intent(in) :: u(n), v(n)
        double precision :: p
        p = dot_product(u, v)
    end function

    ! Producto vectorial entre dos vectores u, v de longitud 3
    function producto_vectorial(u, v) result(w)
        double precision, intent(in) :: u(3), v(3)
        double precision :: w(3)
        w(1) = u(2) * v(3) - u(3) * v(2)
        w(2) = u(3) * v(1) - u(1) * v(3)
        w(3) = u(1) * v(2) - u(2) * v(1)
    end function
end module
```

In [6]:
%%fortran

module vectores
    implicit none
    contains

    ! Producto escalar entre dos vectores u, v de longitud n
    function producto_escalar(n, u, v) result(p)
        integer, intent(in) :: n
        double precision, intent(in) :: u(n), v(n)
        double precision :: p
        p = dot_product(u, v)
    end function

    ! Producto vectorial entre dos vectores u, v de longitud 3
    function producto_vectorial(u, v) result(w)
        double precision, intent(in) :: u(3), v(3)
        double precision :: w(3)
        w(1) = u(2) * v(3) - u(3) * v(2)
        w(2) = u(3) * v(1) - u(1) * v(3)
        w(3) = u(1) * v(2) - u(2) * v(1)
    end function
end module

## Vectores

In [12]:
import numpy as np
u = np.array([1, 2, 3])
v = np.array([1, 0, -1])

vectores.producto_escalar(u, v)

-2.0

In [8]:
vectores.producto_vectorial(u, v)

array([-2.,  4., -2.])

In [13]:
vectores.producto_escalar?

## Compilando manualmente

- Asumiendo que tenemos el mismo código vectores en un archivo llamado `vectores.f90`
- Podemos compilarlo manualmente haciendo:

```bash
$ f2py -c vectores.f90 -m vectores
```
- Esto crea un `vectores<ARCH>.so` o `.pyd`(Windows)

In [16]:
ls fortran/

build/
hola_mundo_sub.cpython-37m-x86_64-linux-gnu.so*
hola_mundo_sub.f90
__pycache__/
setup.py
vectores.cpython-37m-x86_64-linux-gnu.so*
vectores.cpython-38-x86_64-linux-gnu.so*
vectores.f90
vectores.pyf


## Ficheros `*.pyf`

- Para generar módulos Python F2PY auto-genera (y luego elimina)archivos de cabecera de extensión `.pyf`
- Las cabecera que  definen la interfaz a los subprogramas Fortran que se llamarán desde Python.
- Por ejemplo, vamos a generar el fichero de cabecera de nuestro módulo de operaciones vectoriales:

```bash
$ f2py -h vectores.pyf -m vectores vectores.f90
```

> Con el argumento `-h` indicamos cómo queremos que se genere la cabecera, con `-m` el módulo correspondiente y por último incluimos el código fuente. 

## Ficheros `*.pyf`

```fortran
!    -*- f90 -*-
! Note: the context of this file is case sensitive.
python module vectores ! in 
    interface  ! in :vectores
        module vectores ! in :vectores:vectores.f90
            function producto_escalar(n,u,v) result (p) ! in :vectores:vectores.f90:vectores
                integer, optional,intent(in),check(len(u)>=n),depend(u) :: n=len(u)
                double precision dimension(n),intent(in) :: u
                double precision dimension(n),intent(in),depend(n) :: v
                double precision :: p
            end function producto_escalar
            function producto_vectorial(u,v) result (w) ! in :vectores:vectores.f90:vectores
                double precision dimension(3),intent(in) :: u
                double precision dimension(3),intent(in) :: v
                double precision dimension(3) :: w
            end function producto_vectorial
        end module vectores
    end interface 
end python module vectores
! This file was auto-generated with f2py (version:2).
! See http://cens.ioc.ee/projects/f2py2e/
```


## Ficheros `*.pyf`

- La sintaxis de los archivos `.pyf` es muy parecida a la de Fortran, pero no igual. 
- Aveces puede ser útil modificarla para ajustar algunas cosas.
- Por ejemplo, hemos visto antes que F2PY ha transformado el argumento *n* de `producto_escalar` en opcional, y desde Python nunca se utiliza. 
- Se puede ocultar con `intent(hide)` así

```fortran
integer, optional,intent(hide),check(len(u)>=n),depend(u) :: n=len(u)
```

Y lo compilamos con

```bash
$ f2py2 -c vectores.f90 -m vectores vectores.pyf
```
Y si ahora probamos el módulo:

In [14]:
import sys; sys.path.insert(0, "./fortran")
from vectores import vectores

In [15]:
vectores.producto_escalar?

## Directivas

- A veces estos ajustes de la interfaz se pueden incluir en el propio código fuente, sin necesidad de modificar las cabeceras. 
- Las directivas son comentarios en el código Fortran que F2PY puede entender e interpretar. Por ejemplo, si introducimos debajo de la línea 10 de nuestro archivo `vectores.f90`
```fortran
!f2py intent(hide) :: n
```
- `!f2py` para Fortran 95, o `Cf2py` para código FORTRAN 77.

## Empaquetado 

```python
from numpy.distutils.core import setup
from numpy.distutils.core import Extension

EXTENSIONS = [
    Extension(name="vectores",
              sources=["vectores.f90"],
              extra_compile_args=[],
              extra_link_args=[])]

setup(name="fortran_examples",
      ext_modules=EXTENSIONS)
```

- Esto anda $Sii$ usan `numpy.distutils.core.setup()`
- Se compila con

```bash
$ python setup.py build_ext --inplace
```

## Referencias

- https://en.wikipedia.org/wiki/Fortran
- https://pybonacci.org/2013/02/22/integrar-fortran-con-python-usando-f2py/
- https://numpy.org/devdocs/f2py/python-usage.html
- https://gist.github.com/johntut/1d8edea1fd0f5f1c057c